In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import numpy as np
from pathlib import Path

base = Path.cwd().parent
df = pd.read_csv(base / "Data_clean/caract-2005-2023.csv")

df.head()


In [ ]:
# 1) Préparer, valider et filtrer les colonnes `lat` et `long` pour zones françaises
cols = ["lat", "long"]
if not set(cols).issubset(df.columns):
    raise ValueError(f"Le DataFrame `df` doit contenir les colonnes: {cols}")

# Copier et garder les valeurs brutes pour debug
df_loc = df[cols].copy()
for c in cols:
    df_loc[c + '_orig'] = df_loc[c]
    # Remplacer les virgules par des points et nettoyer
    df_loc[c] = df_loc[c].astype(str).str.replace(',', '.').str.strip()
    # Convertir en float; les valeurs non convertibles deviennent NaN
    df_loc[c] = pd.to_numeric(df_loc[c], errors='coerce')

# Définir les bounding boxes pour la France métropolitaine et les territoires d'outre-mer
# (approximations — on peut affiner si besoin)
allowed_boxes = [
    # Métropole
    {"min_lat": 41.0, "max_lat": 51.6, "min_lon": -5.6, "max_lon": 9.8},
    # Guadeloupe
    {"min_lat": 15.6, "max_lat": 16.7, "min_lon": -62.1, "max_lon": -60.8},
    # Martinique
    {"min_lat": 14.3, "max_lat": 14.9, "min_lon": -61.6, "max_lon": -60.8},
    # Guyane
    {"min_lat": 2.0,  "max_lat": 6.5,  "min_lon": -54.6, "max_lon": -50.9},
    # La Réunion
    {"min_lat": -21.5, "max_lat": -20.0, "min_lon": 55.0, "max_lon": 56.2},
    # Mayotte
    {"min_lat": -13.0, "max_lat": -11.5, "min_lon": 43.0, "max_lon": 46.0},
    # Saint-Pierre-et-Miquelon
    {"min_lat": 46.6, "max_lat": 47.1, "min_lon": -56.9, "max_lon": -56.0},
    # Nouvelle-Calédonie
    {"min_lat": -23.0, "max_lat": -18.0, "min_lon": 163.0, "max_lon": 167.5},
    # Polynésie française (approx Tahiti area)
    {"min_lat": -18.8, "max_lat": -14.0, "min_lon": -156.0, "max_lon": -149.0}
]

# Fonction utilitaire pour vérifier si un point est dans une box
def in_allowed(lat, lon, boxes):
    for box in boxes:
        if lat >= box['min_lat'] and lat <= box['max_lat'] and lon >= box['min_lon'] and lon <= box['max_lon']:
            return True
    return False

# Construire un masque initial pour garder uniquement les points dans une de ces boxes
mask_allowed = df_loc.apply(lambda row: in_allowed(row['lat'], row['long'], allowed_boxes) if pd.notnull(row['lat']) and pd.notnull(row['long']) else False, axis=1)

# Tenter d'inverser lat/long pour les points hors zones FR
swapped_idx = []
for idx, row in df_loc[~mask_allowed & df_loc[['lat','long']].notnull().all(axis=1)].iterrows():
    lat, lon = row['lat'], row['long']
    # essayer swap
    if in_allowed(lon, lat, allowed_boxes):
        df_loc.at[idx, 'lat'] = lon
        df_loc.at[idx, 'long'] = lat
        df_loc.at[idx, 'swapped'] = True
        swapped_idx.append(idx)
    else:
        df_loc.at[idx, 'swapped'] = False

# Mettre swapped=False pour les autres
df_loc['swapped'] = df_loc['swapped'].fillna(False)

# Recalculer mask_allowed après swap
mask_allowed = df_loc.apply(lambda row: in_allowed(row['lat'], row['long'], allowed_boxes) if pd.notnull(row['lat']) and pd.notnull(row['long']) else False, axis=1)

# Rapport et inspection
total_rows = len(df_loc)
valid_coords = df_loc[['lat','long']].dropna()
print(f"Total lignes dans df: {len(df)}")
print(f"Lignes avec coords valides après conversion: {len(valid_coords)}")
print(f"Lignes situées dans zones FR définies avant swap: {len([i for i in range(total_rows) if i not in swapped_idx and mask_allowed.iloc[i]])}")
print(f"Lignes corrigées par swap: {len(swapped_idx)}")

# Lignes rejetées (coord valides mais hors zones FR)
rejected = df_loc[~mask_allowed & df_loc[['lat','long']].notnull().all(axis=1)]
print(f"Lignes rejetées (coord valides mais hors zones FR après swap): {len(rejected)}")
if len(rejected) > 0:
    display(rejected[[c + '_orig' for c in cols] + ['lat','long','swapped']].head(10))

# Conserver uniquement les coordonnées valides et dans les zones FR
df_loc = df_loc[mask_allowed & df_loc[['lat','long']].notnull().all(axis=1)].copy().reset_index(drop=True)
print(f"Lignes finales conservées dans df_loc: {len(df_loc)}")

df_loc.head()

In [ ]:
# Belle carte avec heatmap + cluster + mini-map + contrôles de couche
from pathlib import Path
import folium
from folium.plugins import HeatMap, MarkerCluster, MiniMap, Fullscreen

# On part de `df_loc` propre (cellule 2)
if 'df_loc' not in globals():
    raise ValueError("`df_loc` n'existe pas — exécute la cellule de nettoyage avant celle-ci.")

# Échantillonnage pour performance
max_points = 80000
if len(df_loc) > max_points:
    df_sample = df_loc.sample(n=max_points, random_state=1)
else:
    df_sample = df_loc

# Centre et carte
center = [46.5, 2.5]
m = folium.Map(location=center, zoom_start=6, tiles='CartoDB Positron')

# HeatMap layer (plus large et douce)
heat = HeatMap(
    data=df_sample.values.tolist(),
    name='HeatMap',
    min_opacity=0.3,
    max_zoom=12,
    radius=12,
    blur=20,
)
heat.add_to(m)

# MarkerCluster (échantillon réduit pour éviter surcharge)
cluster_sample = df_sample.sample(n=min(2000, len(df_sample)), random_state=2)
mc = MarkerCluster(name='Clusters')
for lat, lon in cluster_sample.values.tolist():
    folium.CircleMarker(location=[lat, lon], radius=3, color='#3388ff', fill=True, fill_opacity=0.6).add_to(mc)
mc.add_to(m)

# MiniMap & Fullscreen
minimap = MiniMap(toggle_display=True)
minimap.add_to(m)
Fullscreen().add_to(m)

# Layer control
folium.LayerControl(collapsed=False).add_to(m)

# Légende simple (HTML/CSS)
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 10px; width:170px; z-index:9999; font-size:14px;">
  <div style="background: white; padding: 10px; border-radius: 6px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
    <b>Heatmap legend</b><br>
    <i style="background: blue; width: 12px; height: 6px; display:inline-block;"></i>&nbsp;Low density<br>
    <i style="background: lime; width: 12px; height: 6px; display:inline-block;"></i>&nbsp;Medium<br>
    <i style="background: orange; width: 12px; height: 6px; display:inline-block;"></i>&nbsp;High<br>
    <i style="background: red; width: 12px; height: 6px; display:inline-block;"></i>&nbsp;Very high
  </div>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save
out_dir = Path.cwd() / 'output'
out_dir.mkdir(parents=True, exist_ok=True)
out_file = out_dir / 'heatmap_france_pretty.html'
m.save(str(out_file))
print(f"Carte prête: {out_file} (points heatmap: {len(df_sample)})")

m

In [ ]:
import pandas as pd
from pathlib import Path
base = Path.cwd().parent
df = pd.read_csv(base / "Data/caract-2023.csv", sep=';')
df

In [ ]:
df["lat"] = df['lat'].astype(str).str.replace(',', '.').str.strip().astype(float)
df['long'] = df['long'].astype(str).str.replace(',', '.').str.strip().astype(float)
df["date"] = pd.to_datetime(
    df["an"].astype(str) + "-" +
    df["mois"].astype(str).str.zfill(2) + "-" +
    df["jour"].astype(str).str.zfill(2),
    format="%Y-%m-%d"
)
df["weekday"] = df["date"].dt.weekday + 1
df_loc = df[["lat", "long"]]

In [ ]:
# Créer une belle carte Folium avec heatmap pour df_loc
from folium.plugins import HeatMap, MiniMap, Fullscreen
import folium

# Vérifier que df_loc existe et contient des données
print(f"Nombre de points dans df_loc: {len(df_loc)}")
print(f"Aperçu des données:")
print(df_loc.head())

# Créer la carte de base avec un style élégant
m = folium.Map(
    location=[46.5, 2.5], 
    zoom_start=6,
    tiles='CartoDB Positron'  # Style épuré et élégant
)

# Nettoyer les données et préparer pour la heatmap
df_clean = df_loc.dropna()
print(f"Points valides pour la heatmap: {len(df_clean)}")

# Utiliser TOUS les points sans échantillonnage
print("Utilisation de tous les points disponibles (pas d'échantillonnage)")
df_sample = df_clean

# Créer la heatmap avec des paramètres optimisés
heat_data = [[row['lat'], row['long']] for idx, row in df_sample.iterrows()]

heatmap = HeatMap(
    heat_data,
    min_opacity=0.2,
    max_zoom=18,
    radius=15,
    blur=25,
    gradient={
        0.2: 'blue',
        0.4: 'cyan', 
        0.6: 'lime',
        0.8: 'yellow',
        1.0: 'red'
    }
)
heatmap.add_to(m)

# Ajouter une mini-carte
minimap = MiniMap(toggle_display=True)
minimap.add_to(m)

# Ajouter le mode plein écran
Fullscreen(
    position='topright',
    title='Mode plein écran',
    title_cancel='Quitter le plein écran'
).add_to(m)

# Ajouter une légende personnalisée
legend_html = '''
<div style="position: fixed; 
            top: 10px; right: 10px; width: 200px; height: 130px; 
            background-color: white; 
            border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);">
    <h4 style="margin-top:0;">Densité des accidents</h4>
    <div style="display: flex; align-items: center; margin: 5px 0;">
        <div style="width: 20px; height: 4px; background: blue; margin-right: 10px;"></div>
        <span>Faible</span>
    </div>
    <div style="display: flex; align-items: center; margin: 5px 0;">
        <div style="width: 20px; height: 4px; background: lime; margin-right: 10px;"></div>
        <span>Moyenne</span>
    </div>
    <div style="display: flex; align-items: center; margin: 5px 0;">
        <div style="width: 20px; height: 4px; background: yellow; margin-right: 10px;"></div>
        <span>Élevée</span>
    </div>
    <div style="display: flex; align-items: center; margin: 5px 0;">
        <div style="width: 20px; height: 4px; background: red; margin-right: 10px;"></div>
        <span>Très élevée</span>
    </div>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Sauvegarder la carte
m.save('heatmap_accidents_france.html')
print("✅ Carte sauvegardée sous 'heatmap_accidents_france.html'")

# Afficher la carte dans le notebook
m